In [1]:
from bottom_up.all import *

%load_ext autoreload

%autoreload 2

dev = [json.loads(_) for _ in open("dev.jsonl")]

In [2]:
# you might miss a few of these if Q is not in the compression
import random
import copy

def pick_at_random(l):
    return random.sample(l, 1)[0]
    
def get_dependents_and_governors(vx, sentence, tree):
    '''add a vertexes children to a queue, sort by prob'''
    children = [d for d in sentence['basicDependencies'] if d["governor"] == vx]
    governor = [d for d in sentence['basicDependencies'] if d["dependent"] == vx][0]
    out = []
    for c in children:
        if c["dependent"] not in tree:
            out.append(c["dependent"])
    if governor["governor"] not in tree:
        out.append(governor["governor"])
    return out

def oracle_path(sentence, pi = pick_at_random):
    T = {i for i in sentence["q"]}
    F = set()
    
    # init frontier
    for v in T:
        for i in get_dependents_and_governors(v, sentence, T):
            F.add(i)
    path = []
    while len(F) > 0:
        v = pi(F)
        if v in sentence["compression_indexes"]:
            for i in get_dependents_and_governors(v, sentence, T):
                F.add(i)
            path.append((T, v, 1))
            T.add(v)
        path.append((copy.deepcopy(T), v, 0))
        F.remove(v)
    
    assert T == set(sentence["compression_indexes"])
        
    return path
        
failures = 0
for d in dev:
    try:
        p =  oracle_path(sentence=d, pi=pick_at_random)
    except:
        failures += 1
        
print failures

106
